# https://github.com/dreamquark-ai/tabnet

In [2]:
from google.colab import drive
drive.mount('./content/')

Mounted at ./content/


In [3]:
!pip install  "git+https://github.com/dreamquark-ai/tabnet.git@develop#egg=pytorch_tabnet" --upgrade
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

  Cloning https://github.com/dreamquark-ai/tabnet.git (to revision develop) to /tmp/pip-install-gx_qav4y/pytorch-tabnet
  Running command git clone -q https://github.com/dreamquark-ai/tabnet.git /tmp/pip-install-gx_qav4y/pytorch-tabnet
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pytorch-tabnet: filename=pytorch_tabnet-3.1.1-cp37-none-any.whl size=39326 sha256=dbf370437df193bcfe8cb3ee932ec034b2624d8b3cea2c676b2862d1c401b1dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-sz7wvvc3/wheels/a5/fe/e1/d7be493728d1fb7284583f5449d4cad80139ee994ef420f522
Successfully built pytorch-tabnet
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11687 sha256=f633493b11ce07d5343c2f8299e737b5817a56f9cdeb7de7fef1017f93f88e6e
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built baye

# Label Encoding

## 전처리

In [1]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
import random
import os
from sklearn.metrics import log_loss
# from bayes_opt import BayesianOptimization
# import lightgbm
import torch
from pytorch_tabnet.tab_model import TabNetClassifier

d = "C:\kaggle_data\credit_card"
lst = os.listdir(d)
print(lst)
train = pd.read_csv(d + '\\' +lst[3])
test = pd.read_csv(d + '\\' +lst[2])
ss = pd.read_csv(d + '\\' +lst[1])
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

# Married, Civil marriage
train['income_per_size'] = np.log(train['income_total']/train['family_size'])
test['income_per_size'] = np.log(test['income_total']/test['family_size'])
train.loc[(train['family_type']=='Married')|(train['family_type']=='Civil marriage'),'income_per_size']\
= train['income_per_size'] * 2

test.loc[(test['family_type']=='Married')|(test['family_type']=='Civil marriage'),'income_per_size']\
= test['income_per_size'] * 2

def simple_marry(x):
    if x == 'Married' or x =='Civil marriage':
        return '0'
    elif x == 'Separated' or x == 'Widow':
        return '1'
    else:
        return '2'

for df in [train,test]:
    df['family_bins'] = df['family_type'].apply(simple_marry)

# income_total을 로그변환
train['income_total'] = np.log(train['income_total'])
# train = train.drop('income_total',1)
test['income_total'] = np.log(test['income_total'])
# test = test.drop('income_total',1)

# car와 reality를 합친 새로운 칼럼 careality
train['car'] =train['car'].apply(lambda x: int(x=='Y'))
train['reality'] =train['reality'].apply(lambda x: int(x=='Y'))
test['car'] =test['car'].apply(lambda x: int(x=='Y'))
test['reality'] =test['reality'].apply(lambda x: int(x=='Y'))

train['careality'] = train['car'] + train['reality']
train = train.drop(['car', 'reality'],1)

test['careality'] = test['car'] + test['reality']
test = test.drop(['car', 'reality'],1)

train.drop('gender',1,inplace=True)
test.drop('gender',1,inplace=True)

object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)
###############################
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)
########################################




## 제곱
for df in [train,test]:
    df['income_per_size'] = df['income_per_size'].apply(lambda x: x**2)
    
c = 'income_per_size'
mean = train[c].mean()
std = train[c].std()

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

#####################
c = 'income_total'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

#####################
c = 'begin_month'
k = 2.2

train[c] = train[c].apply(lambda x: x**2)
test[c] = test[c].apply(lambda x: x**2)

mean = train[c].mean()
std = train[c].std()

idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
# train = train.drop(idxs).reset_index(drop=True)

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

######################
c = 'DAYS_BIRTH'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

#기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std



###################
c = 'DAYS_EMPLOYED'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)



ran = ['child_num', 'DAYS_BIRTH', 'DAYS_EMPLOYED','family_size', 'begin_month', 'income_per_size']
from sklearn.preprocessing import QuantileTransformer
tf = QuantileTransformer(n_quantiles=100,random_state=42, output_distribution='normal')
tf.fit(train[ran])
train[ran] = tf.transform(train[ran])
test[ran] = tf.transform(test[ran])



MAX_EPOCHS = 1000
BATCH_SIZE = 1024
VIRTUAL_BATCH_SIZE = 128
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Using {}".format(DEVICE))

['.ipynb_checkpoints', 'sample_submission.csv', 'test.csv', 'train.csv', 'Untitled.ipynb']
Using cuda


In [2]:
# cat_idxs = [ i for i, f in enumerate(train.columns) if f in object_col]

# cat_dims = [len(total.iloc[:,idx].unique()) for idx in cat_idxs]

In [3]:
n = 10
skf = StratifiedKFold(n_splits=n, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))
random.seed(42)

- n_step: 4,6

In [ ]:
from tqdm import tqdm
# loss=0
# models={}
# historys={}
results = []
for n_step in [5,6,9]:
    for ind in range(1,6):
        for sha in range(1,6):
            loss=0
            models={}
            historys={}
            for fold in tqdm(range(n)):
                train_idx, valid_idx = folds[fold]
                X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                                  train['credit'][train_idx].values, train['credit'][valid_idx].values 
                clf = TabNetClassifier(n_d=9, n_a=9,
                                n_steps=n_step,
                                gamma=1.006,
                                n_independent=ind,
                                n_shared=sha,
            #                     momentum=0.3, 
            #                     clip_value=2., 
                                lambda_sparse=0.01994,
                                seed=42,

                                optimizer_fn=torch.optim.Adam,
                                optimizer_params=dict(lr=1e-2), 
                                scheduler_params = {"gamma": 0.95,
                                  "step_size": 20},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
                                device_name = 'auto',
                                verbose=0)

                history = clf.fit(
                  X_train=X_train, y_train=y_train,
                  eval_set=[(X_valid, y_valid)],
                  eval_metric=['logloss'],
                  max_epochs=MAX_EPOCHS ,
                  patience=50, # please be patient ^^
                  batch_size=32768,
                  virtual_batch_size=16384,
                  num_workers=0)
                models[fold] = clf
                historys[fold]= history
                # y_pred = clf.predict_proba(X_valid.values)
                y_pred = clf.predict_proba(X_valid)
                loss += log_loss(y_valid,y_pred)
                results.append([n_step,ind,sha])
                print(n_step,ind,sha)

  0%|                                            | 0/10 [00:00<?, ?it/s]


Early stopping occurred at epoch 207 with best_epoch = 157 and best_val_0_logloss = 0.83234
Best weights from best epoch are automatically used!


 10%|███▌                                | 1/10 [01:00<09:08, 60.95s/it]

5 1 1

Early stopping occurred at epoch 232 with best_epoch = 182 and best_val_0_logloss = 0.82777
Best weights from best epoch are automatically used!


 20%|███████▏                            | 2/10 [02:05<08:24, 63.03s/it]

...


Early stopping occurred at epoch 269 with best_epoch = 219 and best_val_0_logloss = 0.80763
Best weights from best epoch are automatically used!
6 2 2


 20%|███████▏                            | 2/10 [02:41<10:19, 77.43s/it]


Early stopping occurred at epoch 166 with best_epoch = 116 and best_val_0_logloss = 0.81825
Best weights from best epoch are automatically used!
6 2 2

Early stopping occurred at epoch 126 with best_epoch = 76 and best_val_0_logloss = 0.85921
Best weights from best epoch are automatically used!


 30%|██████████▊                         | 3/10 [03:28<07:25, 63.65s/it]

6 2 2


In [18]:
train.columns

Index(['child_num', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'family_size',
       'begin_month', 'credit', 'income_per_size', 'careality',
       'income_type_Commercial associate', 'income_type_Pensioner',
       'income_type_State servant', 'income_type_Student',
       'income_type_Working', 'edu_type_Academic degree',
       'edu_type_Higher education', 'edu_type_Incomplete higher',
       'edu_type_Lower secondary', 'edu_type_Secondary / secondary special',
       'family_type_Civil marriage', 'family_type_Married',
       'family_type_Separated', 'family_type_Single / not married',
       'family_type_Widow', 'house_type_Co-op apartment',
       'house_type_House / apartment', 'house_type_Municipal apartment',
       'house_type_Office apartment', 'house_type_Rented apartment',
       'house_type_With parents', 'occyp_type_Accountants',
       'occyp_type_Cleaning staff', 'occyp_type_Cooking staff',
       'occyp_type_Co

In [16]:
loss/10

0.8033376689532075

In [14]:
models[0]

TabNetClassifier(n_d=9, n_a=9, n_steps=5, gamma=1.006, cat_idxs=[], cat_dims=[], cat_emb_dim=1, n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.01994, seed=42, clip_value=1, verbose=0, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.01}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'gamma': 0.95, 'step_size': 20}, mask_type='sparsemax', input_dim=55, output_dim=3, device_name='auto')

## drdrdr 모델링, 탐색

In [8]:
##### 파라미터 최적화 탐색 중
# n_d, n_a == 9, 16
# lambda_sparse == 0.0001, 0.1478, 0.01994 (9일 떄)
# decision_step == 
##### 
n = 4
skf = StratifiedKFold(n_splits=n, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))
random.seed(42)

max_epochs = 1000
models={}
def eval_tabnet(n_independent):
  loss=0
  for fold in range(n):
      train_idx, valid_idx = folds[fold]
      X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                          train['credit'][train_idx].values, train['credit'][valid_idx].values 
      
      clf = TabNetClassifier(n_d=9, n_a=9,
                        n_steps=5,
                        gamma=1.006,
#                         n_independent=int(n_independent),
                        #  n_shared=int(n_shared),
                        # momentum=0.3, 
                        # clip_value=2., 
                             
                        lambda_sparse=0.01994,
                        seed=42,
                        optimizer_fn=torch.optim.Adam,
                        optimizer_params=dict(lr=1e-2), 
                        scheduler_params = {"gamma": 0.95,
                          "step_size": 20},
                        scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
                        device_name = 'auto',
                        verbose=0)
      
      clf.fit(
          X_train=X_train, y_train=y_train,
          eval_set=[(X_valid, y_valid)],
      #     eval_name=None,
          eval_metric=['logloss'],
      #     loss_fn=None,
          max_epochs=max_epochs ,
          patience=50, # please be patient ^^
          batch_size=1024,
          virtual_batch_size=256,
          num_workers=1,
          )
      models[fold] = clf
      # y_pred = clf.predict_proba(X_valid.values)
      y_pred = clf.predict_proba(X_valid)
      loss += log_loss(y_valid,y_pred)
  return -loss/n

n_d9일떄 gamma찾는 중

In [ ]:
lgbBO = BayesianOptimization(eval_tabnet, {'n_independent': (1,5),
                                        # 'n_d': (9,16)
                                        # 'min_data_in_leaf': (100, 2000),
                                        
                                          })
lgbBO.maximize(n_iter=20, init_points=2)

|   iter    |  target   | n_inde... |
-------------------------------------

Early stopping occurred at epoch 168 with best_epoch = 118 and best_val_0_logloss = 0.80698
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 168 with best_epoch = 118 and best_val_0_logloss = 0.80577
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 107 with best_epoch = 57 and best_val_0_logloss = 0.81582
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 328 with best_epoch = 278 and best_val_0_logloss = 0.80272
Best weights from best epoch are automatically used!
|  1        | -0.8078   |  3.593    |

Early stopping occurred at epoch 124 with best_epoch = 74 and best_val_0_logloss = 0.81269
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 122 with best_epoch = 72 and best_val_0_logloss = 0.81283
Best weights from best epoch are automatically used!

Early stoppi

In [15]:
models={}
for fold in range(n):
  print(n+1,'번째')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                      train['credit'][train_idx].values, train['credit'][valid_idx].values 
  
  clf = TabNetClassifier(n_d=int(9), n_a=int(9),
                    # n_steps=3,
                    # gamma=1.0,# n_independent=2, n_shared=2,
                    # lambda_sparse=1e-4,
                    cat_idxs=cat_idxs,
                      cat_dims=cat_dims,
                      cat_emb_dim=1,
                    seed=42,
                    # momentum=0.3, 
                    # clip_value=2., 
                    optimizer_fn=torch.optim.Adam,
                    optimizer_params=dict(lr=1e-2), 
                    scheduler_params = {"gamma": 0.95,
                      "step_size": 20},
                    scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
                    device_name = 'auto',
                    verbose=0)
  clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_valid, y_valid)],
    #     eval_name=None,
        eval_metric=['logloss'],
    #     loss_fn=None,
        max_epochs=max_epochs ,
        patience=50, # please be patient ^^
        batch_size=1024,
        virtual_batch_size=256,
        num_workers=1)
  models[fold] = clf

6 번째


KeyboardInterrupt: ignored

In [16]:
clf = TabNetClassifier(n_d=int(9), n_a=int(9),
                  # n_steps=3,
                  # gamma=1.0,# n_independent=2, n_shared=2,
                  # lambda_sparse=1e-4,
                  cat_idxs=cat_idxs,
                    cat_dims=cat_dims,
                    cat_emb_dim=1,
                  seed=42,
                  # momentum=0.3, 
                  # clip_value=2., 
                  optimizer_fn=torch.optim.Adam,
                  optimizer_params=dict(lr=1e-2), 
                  scheduler_params = {"gamma": 0.95,
                    "step_size": 20},
                  scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
                  device_name = 'auto',
                  verbose=0)
clf.fit(
      X_train=X_train, y_train=y_train,
      eval_set=[(X_valid, y_valid)],
  #     eval_name=None,
      eval_metric=['logloss'],
  #     loss_fn=None,
      max_epochs=max_epochs ,
      patience=50, # please be patient ^^
      batch_size=1024,
      virtual_batch_size=256,
      num_workers=1)


Early stopping occurred at epoch 114 with best_epoch = 64 and best_val_0_logloss = 0.87283
Best weights from best epoch are automatically used!


In [17]:
preds = clf.predict_proba(test.values)
preds

array([[0.19809672, 0.08804115, 0.7138621 ],
       [0.24417205, 0.07967963, 0.6761483 ],
       [0.13098761, 0.17606707, 0.69294536],
       ...,
       [0.11808927, 0.12027843, 0.76163226],
       [0.06839902, 0.15826786, 0.7733332 ],
       [0.12993054, 0.3628431 , 0.5072264 ]], dtype=float32)

In [18]:
ss.iloc[:,1:]=0
ss.iloc[:,1:] += preds
ss

,index,0,1,2
0,26457,0.198097,0.088041,0.713862
1,26458,0.244172,0.079680,0.676148
2,26459,0.130988,0.176067,0.692945
3,26460,0.059098,0.189698,0.751205
4,26461,0.186102,0.133101,0.680797
...,...,...,...,...
9995,36452,0.169928,0.310359,0.519713
9996,36453,0.267642,0.247663,0.484696
9997,36454,0.118089,0.120278,0.761632
9998,36455,0.068399,0.158268,0.773333


In [ ]:
ss.iloc[:,1:]=0
for fold in range(n):
    ss.iloc[:,1:] += models[fold].predict_proba(test.values)/n

In [19]:
ss.to_csv('tabnet_test.csv', index=False) # 0.7272812144